In [0]:
from google.colab import drive
drive.mount("/MyDrive")

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.svm import SVC

In [0]:
iris = datasets.load_iris()

X = iris.data[:, :2]
y = iris.target
plt.scatter (X[:, 0], X[:, 1], c = iris.target)
plt.title("Iris data")
plt.xlabel("sepal_length")
plt.ylabel("sepal_width")

In [0]:
iris = datasets.load_iris()

X = iris.data
y = iris.target

C_range =  [1.,3.,5.,10.,40.,60.,80.,100.]
# C_range = np.arange(1.,100.)
gamma_range = np.logspace(-9, 0, 10)
# gamma_range = np.arange(1.,100.)
param_grid = dict(gamma=gamma_range, C=C_range)
param_grid_ = dict(C=C_range)

degree_range = np.arange(1,10)

cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv)
grid_ = GridSearchCV(SVC(kernel="linear"),param_grid=param_grid_, cv=cv)
grid_a = GridSearchCV(SVC(kernel="poly"),param_grid=param_grid, cv=cv)

grid_a.fit(X, y)
grid_.fit(X, y)
grid.fit(X, y)

print("The linear kernel`s best parameters are %s" % (grid_.best_params_))
print("The rbf kernel`s best parameters are %s" % (grid.best_params_))
print("The polynomial kernel`s best parameters are %s" % (grid_a.best_params_))


In [0]:
C = []
gamma = []

X = X[:, :2]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

for this_C in C_range:
    clf = SVC(kernel='linear',C=this_C).fit(X_train,y_train)
    scoretrain = clf.score(X_train, y_train)
    scoretest = clf.score(X_test, y_test)
    C.append(scoretest)
    print("Linear SVM value of C:{}, Test Score: {:2f}\n".format(this_C,scoretrain,scoretest))

C = C_range[C.index(max(C))]
print("C: ",C)

In [0]:
C = 0
gamma = 0
temp = 0
for this_gamma in gamma_range:
    for this_C in C_range:
        clf3 = SVC(kernel='rbf',C=this_C,gamma=this_gamma).fit(X_train,y_train)
        clf3train = clf3.score(X_train,y_train)
        clf3test  = clf3.score(X_test,y_test)
        if clf3test > temp:
          temp = clf3test
          C = this_C
          gamma = this_gamma
        print("SVM for Non Linear \n C:{} Gamma: {} Test Score : {:2f}\n".format(this_C,this_gamma,clf3test))
        
print("C: {} gamma: {} Test Score : {:2f}".format(C, gamma, temp))

In [0]:
C = 0
gamma = 0
temp = 0
for this_gamma in gamma_range:
    for this_C in C_range:
        clf3 = SVC(kernel='poly',C=this_C,gamma=this_gamma).fit(X_train,y_train)
        clf3train = clf3.score(X_train,y_train)
        clf3test  = clf3.score(X_test,y_test)
        if clf3test > temp:
          temp = clf3test
          gamma = this_gamma
          C = this_C
        print("SVM for Polynomial \n C:{} Gamma: {} Test Score : {:2f}\n".format(this_C,this_gamma,clf3test))
print("C: {} gamma: {} Test Score: {:2f}".format(C, gamma,temp))

In [0]:
def make_meshgrid(x, y, h=0.2):
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy =np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    
    return xx, yy
def plot_contours(ax, clf, xx, yy, **params):
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

In [0]:
X = iris.data[:, :2]
y = iris.target

models = (svm.SVC(kernel='linear', C=1.),
         svm.LinearSVC(C=1., max_iter=10000),
         svm.SVC(kernel='rbf', gamma=.1, C=3.),
         svm.SVC(kernel='poly',degree=3,gamma=.01,C=40))
models = (clf.fit(X, y) for clf in models)

titles = ('SVC with linear kernel',
          'LinearSVC (linear kernel)',
          'SVC with RBF kernel',
          'SVC with polynomial (degree 3) kernel')

fig, sub = plt.subplots(2, 2)
plt.subplots_adjust(wspace=.4, hspace=.4)

X0, X1 = X[:, 0], X[:, 1]
xx, yy = make_meshgrid(X0, X1)

for clf, title, ax in zip(models, titles, sub.flatten()):
    plot_contours(ax, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
    ax.scatter(X0, X1, c=y, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xlabel('Sepal length')
    ax.set_ylabel('Sepal width')
    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_title(title)

plt.show()